# RAG basics with LangChain and a pdf

- Author: Martin Fockedey with the help of copilot
- This is based on [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)


## Environment Setup

Set up the environment for using Mistral AI with LangChain.

**[Note]**
- You have a Mistral API key on your Teams group channel
- Store your API key in a `.env` file as `MISTRAL_API_KEY`

In [20]:
%pip install -q python-dotenv langchain_mistralai langchain_community langchain_text_splitters langchain_core faiss-cpu pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\FKY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
# Configuration file to manage the API KEY as an environment variable
from dotenv import load_dotenv

# Load API KEY information
load_dotenv(override=True)

True

## RAG Basic Pipeline with Mistral

Below is the implementation of the RAG pipeline using Mistral AI. We'll use:
- **Mistral AI** for language model generation
- **Mistral Embeddings** for document embeddings
- **FAISS** for vector storage
- **PyMuPDF** for PDF loading

The content of each module can be adjusted to fit specific scenarios, allowing for iterative improvement of the structure to suit your documents.

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

### Step 1: Load Documents

Load the PDF document using PyMuPDFLoader.

In [23]:
# Step 1: Load Documents
loader = PyMuPDFLoader("RéglementECAM.pdf")
docs = loader.load()
print(f"Number of pages in the document: {len(docs)}")

Number of pages in the document: 61


Print the content of a sample page.

In [24]:
print(docs[10].page_content)

Haute Ecole ICHEC – ECAM – ISFSC - Règlement des études 2025-2026 
11 
 
• 
Soit par courrier électronique à l’adresse renseignée par l’étudiante ou l’étudiant dans 
son dossier de demande d’admission. 
L’étudiante ou l’étudiant peut alors introduire un recours contre cette décision auprès du 
Commissaire du gouvernement, selon les modalités fixées à l’article 104 §1 du présent 
règlement.  
En l’absence de réponse à sa demande d’admission au 31 octobre, l’étudiante ou l’étudiant peut 
également introduire un recours auprès du Commissaire du gouvernement 
Chapitre 4 : Finançabilité des étudiants 
Article 12 : Conditions de finançabilité 
L’étudiante ou l’étudiant devra satisfaire aux conditions suivantes pour être finançable :   
- 
d’une part, être de nationalité d’un Etat membre de l’Union européenne ou satisfaire 
à une des conditions prévues à l’article 3, 1° à 3,7° du « Décret finançabilité » ; 
 
- 
et d’autre part, satisfaire sur le plan académique au moins, à une des conditions

Check the metadata of the document.

In [25]:
docs[10].__dict__

{'id': None,
 'metadata': {'source': 'RéglementECAM.pdf',
  'file_path': 'RéglementECAM.pdf',
  'page': 10,
  'total_pages': 61,
  'format': 'PDF 1.7',
  'title': 'Vade Mecum',
  'author': 'MD',
  'subject': '',
  'keywords': '',
  'creator': 'Microsoft® Word pour Microsoft\xa0365',
  'producer': 'Microsoft® Word pour Microsoft\xa0365',
  'creationDate': "D:20250912135321+02'00'",
  'modDate': "D:20250912135321+02'00'",
  'trapped': ''},
 'page_content': 'Haute Ecole ICHEC – ECAM – ISFSC - Règlement des études 2025-2026 \n11 \n \n• \nSoit par courrier électronique à l’adresse renseignée par l’étudiante ou l’étudiant dans \nson dossier de demande d’admission. \nL’étudiante ou l’étudiant peut alors introduire un recours contre cette décision auprès du \nCommissaire du gouvernement, selon les modalités fixées à l’article 104 §1 du présent \nrèglement.  \nEn l’absence de réponse à sa demande d’admission au 31 octobre, l’étudiante ou l’étudiant peut \négalement introduire un recours auprès 

### Step 2: Split Documents

Split the documents into smaller chunks for better retrieval and processing.

In [26]:
# Step 2: Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"Number of split chunks: {len(split_documents)}")

Number of split chunks: 497


### Step 3: Generate Embeddings

Use Mistral AI embeddings to convert text chunks into vector representations.

**Note**: Mistral AI provides the `mistral-embed` model for embeddings.

In [27]:
# Step 3: Generate Embeddings
embeddings = MistralAIEmbeddings(model="mistral-embed")

### Step 4: Create and Save the Database

Create a FAISS vector store from the document chunks and embeddings.

In [28]:
# Step 4: Create and Save the Database
# Create a vector store
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
print("Vector store created successfully!")

Vector store created successfully!


Test the vector store with a similarity search.

In [29]:
# Test similarity search
for doc in vectorstore.similarity_search("Financabilité d'un étudiant"):
    print(doc.page_content)
    print("---")

La finançabilité est donc à étudier en regard de chaque parcours individuel et afin 
de préserver un maximum de clarté dans ce règlement, nous invitons les étudiants 
qui souhaitent une information complète sur ces règles de finançabilité à 
consulter la version coordonnée du « Décret Finançabilité », disponible via le lien 
suivant : https://gallilex.cfwb.be 
 
Par ailleurs, l’étudiante ou l’étudiant qui souhaite s’assurer de sa finançabilité peut
---
- 
Soit contester formellement sa non-finançabilité, en motivant alors juridiquement les 
raisons qui lui donnent la conviction qu’elle/qu’il est bien finançable, dans le respect des 
dispositions du « Décret finançabilité ». 
Tout recours qui remet en cause l’état de non-finançabilité sera préalablement examiné 
par le Commissaire du Gouvernement, qui remettra un avis à la Haute Ecole quant au 
financement de l’étudiante ou de l’étudiant et sur base duquel la commission prendra
---
également consulter la page web interactive de la Fédér

### Step 5: Create Retriever

Create a retriever to search and retrieve information from the vector store.

In [30]:
# Step 5: Create Retriever
# Search and retrieve information contained in the documents
retriever = vectorstore.as_retriever()

Send a query to the retriever and check the resulting chunks.

In [31]:
# Test the retriever
results = retriever.invoke("Quelle est la punition pour le plagiat?")
for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(result.page_content)


Result 1:
En cas de plagiat avéré, le jury n’aura pas à apporter la preuve de l’intention de frauder. 
 
Toute citation devra être placée entre guillemets et être assortie de la référence précise de la 
source. 
 
Les normes de citation des sources en vigueur dans chaque département sont mises à la 
disposition de l’étudiante ou de l’étudiant, selon les modalités internes au département. 
 
L’étudiante ou l’étudiant pourra se voir imposer le dépôt de la version électronique de son travail.

Result 2:
l’étudiant sera poursuivi pour plagiat, qui pourra ; en fonction de son degré de gravité et/ou de 
son caractère délibérément frauduleux apprécié par la Direction du Département, être assimilé 
à une faute grave ou à une fraude à l’évaluation et donner lieu aux sanctions prévues dans ce 
cadre (cf. articles 87, 88 et 100 du présent règlement). 
3ème Partie : Clauses particulières 
Article 97 : Associations ou groupements d’étudiants

Result 3:
si la Direction du Département estime, sur ba

### Step 6: Create Prompt

Create a prompt template that will use the retrieved context to answer questions.

In [32]:
# Step 6: Create Prompt
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

### Step 7: Setup LLM

Initialize the Mistral AI language model. We'll use `mistral-small-latest` for cost-effective performance.

In [33]:
# Step 7: Setup LLM
llm = ChatMistralAI(model="mistral-small-latest", temperature=0)

### Step 8: Create Chain

Create the complete RAG chain using LCEL (LangChain Expression Language).

In [34]:
# Step 8: Create Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Test the RAG Chain

Input a query (question) into the created chain and execute it.

In [36]:
# Run Chain
# Input a query about the document and print the response
question = "Where has the application of AI in healthcare been confined to so far?"
response = chain.invoke(question)
print(response)
print("-----")
question = "Qu'est-ce que je risque à plagier un travail académique?"
response = chain.invoke(question)
print(response)

I don't know. The provided context does not contain any information about the application of AI in healthcare.
En cas de plagiat avéré, vous risquez des sanctions, car le jury n'a pas besoin de prouver l'intention de frauder. Les conséquences peuvent varier en fonction de la gravité du plagiat et de son caractère frauduleux, évalué par la Direction du Département. Cela peut être assimilé à une faute grave ou à une fraude à l'évaluation, entraînant des sanctions prévues dans le règlement (articles 87, 88 et 100). Les sanctions peuvent inclure des mesures disciplinaires, voire des conséquences académiques plus sévères.

Si vous avez d'autres questions ou besoin de précisions, n'hésitez pas à demander.
En cas de plagiat avéré, vous risquez des sanctions, car le jury n'a pas besoin de prouver l'intention de frauder. Les conséquences peuvent varier en fonction de la gravité du plagiat et de son caractère frauduleux, évalué par la Direction du Département. Cela peut être assimilé à une faute

## Complete Code

This is a combined code that integrates steps 1 through 8 into a single executable block.

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

# Step 1: Load Documents
loader = PyMuPDFLoader("RéglementECAM.pdf")
docs = loader.load()

# Step 2: Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# Step 3: Generate Embeddings
embeddings = MistralAIEmbeddings(model="mistral-embed")

# Step 4: Create and Save the Database
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# Step 5: Create Retriever
retriever = vectorstore.as_retriever()

# Step 6: Create Prompt
prompt = PromptTemplate.from_template(
    """You are an assistant for the school ECAM-ICHEC-ISFSC for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# Step 7: Load LLM
llm = ChatMistralAI(model="mistral-small-latest", temperature=0)

# Step 8: Create Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG pipeline created successfully!")

An error occurred with MistralAI: Illegal header value b'Bearer '


LocalProtocolError: Illegal header value b'Bearer '

In [4]:
# Run Chain in a while loop
# Input a query about the document and print the response 
# pay attention the input function in vs code cell open a pop up on the top of the screen
while True:
    question = input("Enter your question: ")
    response = chain.invoke(question)
    print(response)

Je ne sais pas. L'information concernant l'ECAM n'est pas présente dans le contexte fourni.
Je ne sais pas. Le contexte fourni ne mentionne pas ce qu'est un "recours".
Je ne sais pas. Le contexte fourni ne mentionne pas ce qu'est un "recours".
Je ne sais pas. Le contexte fourni ne contient pas d'informations sur la procédure pour faire un recours à un examen.
Je ne sais pas. Le contexte fourni ne contient pas d'informations sur la procédure pour faire un recours à un examen.
It seems like you haven't provided a specific question to answer. Could you please provide the question you'd like me to address based on the given context?
It seems like you haven't provided a specific question to answer. Could you please provide the question you'd like me to address based on the given context?


KeyboardInterrupt: Interrupted by user